In [100]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F

In [101]:
spark = SparkSession.builder \
    .master("local[*]")\
    .appName("test")\
    .getOrCreate()

In [102]:
spark

In [103]:
# get data schema

import pandas as pd
df_pandas = pd.read_csv("./fhv_tripdata_2019-10.csv/fhv_tripdata_2019-10.csv")



In [104]:
spark.createDataFrame(df_pandas.head()).schema

c:\Users\ASUS\Python38\lib\site-packages\pyspark\sql\pandas\conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\ASUS\Python38\lib\site-packages\pyspark\sql\pandas\conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', DoubleType(), True), StructField('DOlocationID', DoubleType(), True), StructField('SR_Flag', DoubleType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [105]:
schema = types.StructType(
    [types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.StringType(), True), 
    types.StructField('dropOff_datetime', types.StringType(), True), 
    types.StructField('PUlocationID',types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)])

In [106]:
# read csv file
df = spark.read\
    .option("header", "true")\
    .schema(schema)\
    .csv("./fhv_tripdata_2019-10.csv/fhv_tripdata_2019-10.csv")

In [107]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:56:29|2019-10-01 00:57:47|         264|         264|   null|                B00014|
|              B00014|2019-10-01 00:23:09|2019-10-01 00:28:27|         264|         264|   null|                B00014|
|     B00021         |2019-10-01 00:00:4

In [108]:
#create partition table
df = df.repartition(6)
df.write.parquet("fhvhv/2019/10/", mode="overwrite")

In [109]:
#read from partition
df = spark.read.parquet("fhvhv/2019/10/")

In [110]:
new_df = df\
    .withColumn('pickup_date', F.to_date(df.pickup_datetime))\
    .withColumn('dropoff_date', F.to_date(df.dropOff_datetime))\
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')

In [111]:
new_df.show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2019-10-02|  2019-10-02|         264|         264|
| 2019-10-01|  2019-10-01|         264|         264|
| 2019-10-02|  2019-10-02|          42|         243|
| 2019-10-02|  2019-10-02|         264|          70|
| 2019-10-03|  2019-10-03|         264|         264|
| 2019-10-02|  2019-10-02|         162|         265|
| 2019-10-02|  2019-10-02|         264|         126|
| 2019-10-02|  2019-10-02|         264|           3|
| 2019-10-01|  2019-10-01|         264|         221|
| 2019-10-01|  2019-10-01|         264|         254|
| 2019-10-02|  2019-10-02|         264|          75|
| 2019-10-01|  2019-10-01|         264|         264|
| 2019-10-02|  2019-10-02|         264|          91|
| 2019-10-01|  2019-10-01|         264|          18|
| 2019-10-01|  2019-10-01|         264|         173|
| 2019-10-03|  2019-10-03|         264|       

# Count records on 15th October

In [112]:

new_df\
    .where(F.col("pickup_date")=="2019-10-15")\
    .count()

62610

In [113]:
df.show()
    

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B03201|2019-10-02 06:50:00|2019-10-02 08:08:00|         264|         264|   null|                B02867|
|              B01129|2019-10-01 09:54:00|2019-10-01 10:09:00|         264|         264|   null|                B01129|
|              B03142|2019-10-02 09:10:00|2019-10-02 09:37:45|          42|         243|   null|                B02884|
|              B01381|2019-10-02 05:26:55|2019-10-02 05:34:34|         264|          70|   null|                B01381|
|              B02285|2019-10-03 07:30:00|2019-10-03 09:46:00|         264|         264|   null|                B02285|
|              B02715|2019-10-02 20:37:3

# The longest trip

In [114]:
# Convert string columns to timestamp columns
df = df.withColumn("pickup_time", F.col("pickup_datetime").cast("timestamp"))
df = df.withColumn("dropoff_time", F.col("dropoff_datetime").cast("timestamp"))

In [115]:
# Calculate the duration column
df = df.withColumn("duration", (F.col("dropoff_time").cast("long") - F.col("pickup_time").cast("long")))

In [116]:
# Find the row with the maximum duration
max_duration_row = df.agg((F.max("duration") / 3600).alias('longest_trip_hour'))
print(f"longest trip: {max_duration_row.collect()[0]['longest_trip_hour']} hours")

longest trip: 631152.5 hours


# Least frequent pickup location zone

In [117]:
df_zone = spark.read\
    .option("header", "true")\
    .csv("./taxi_zone_lookup.csv")

In [118]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [126]:
full_join_df = df.join(df_zone, df['PUlocationID'] == df_zone['LocationID'], "inner")

In [127]:
full_join_df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------------+-------------------+--------+----------+---------+--------------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|        pickup_time|       dropoff_time|duration|LocationID|  Borough|                Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------------+-------------------+--------+----------+---------+--------------------+------------+
|              B03201|2019-10-02 06:50:00|2019-10-02 08:08:00|         264|         264|   null|                B02867|2019-10-02 06:50:00|2019-10-02 08:08:00|    4680|       264|  Unknown|                  NV|         N/A|
|              B01129|2019-10-01 09:54:00|2019-10-01 10:09:00|         264|         264|   null|        

In [135]:
count_df = full_join_df\
    .groupBy("Zone")\
    .count()

In [143]:
least_PU_location = count_df\
    .orderBy(F.col("count"))\
    .first()

print(f"least pickup location: {least_PU_location['Zone']}")

least pickup location: Jamaica Bay
